In [2]:
!pip install tiktoken

   ---------------------------------------- 0.0/884.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/884.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/884.5 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/884.5 kB ? eta -:--:--
   ---------------------- --------------- 524.3/884.5 kB 932.9 kB/s eta 0:00:01
   ----------------------------------- ---- 786.4/884.5 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 884.5/884.5 kB 1.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import tiktoken

# Define the pricing per 1,000 tokens for the chosen model
PRICE_PER_1000_TOKENS = 0.00003  # Replace with the correct value for GPT-4
MAX_OUTPUT_TOKENS = 16  # 1 token for grade + 15 tokens for reason

# Tokenizer setup for gpt-4
tokenizer = tiktoken.encoding_for_model("gpt-4")

def count_input_tokens(prompt, post):
    """
    Counts the number of input tokens for the given prompt and post.
    """
    formatted_prompt = f"{prompt}\n\nThe post is: {post}"
    input_tokens = len(tokenizer.encode(formatted_prompt))
    return input_tokens

def estimate_cost(sheet_df, prompts):
    """
    Estimates token usage and cost for a sheet based on input and output tokens.
    """
    total_tokens_used = 0

    for idx, row in sheet_df.iterrows():
        post = row['Art Post']  # Replace 'Post' with the actual column name containing posts
        if pd.notnull(post):  # Skip empty posts
            # Calculate input tokens for the first prompt
            input_tokens1 = count_input_tokens(prompts[0], post)
            total_tokens_used += input_tokens1 

            # Calculate input tokens for the second prompt
            input_tokens2 = count_input_tokens(prompts[1], post)
            total_tokens_used += input_tokens2 

    return total_tokens_used

# Load the Excel file


# Define the two prompts
prompts = [
    """You are a serious crowdsourcing human worker. Grade how productive the author of each post seems to be doing, according to their related experience. 
    The grade should be exactly one of these grades: rather productive, rather neutral, rather not productive, subject has rather nothing to do with productivity, author not a producer himself. 
    Please reply with the format:
    Grade: grade,
    Reason: reason (up to 15 words)""",
    
    """You are a serious crowdsourcing human worker; grade how hard and painful it is, for the author of each post to create, according to his related experience. 
    The grade should be exactly one of these grades: rather hard to create, rather neutral to creation, rather not hard to create, subject has rather nothing to do with creativity, author not a creator himself. 
    Please reply with the format:
    Grade: grade,
    Reason: reason (up to 15 words)"""
]

# Process each sheet and calculate total cost
total_tokens_used = 0
file_path = "user_posts_art_and_sport_only_sport_filter_min_diff.xlsx"
art_and_sport_file = pd.read_excel(file_path)
art_and_sport_file = art_and_sport_file.head(50)
tokens_used = estimate_cost(art_and_sport_file, prompts)
total_tokens_used += tokens_used

file_path = "only_art_users.xlsx"
only_art_file = pd.read_excel(file_path)
only_art_file = only_art_file.head(50)
tokens_used = estimate_cost(only_art_file, prompts)
total_tokens_used += tokens_used

# Calculate cost
total_cost = total_tokens_used  * PRICE_PER_1000_TOKENS

# Print results
print(f"Estimated total input tokens used: {total_tokens_used}")
print(f"Estimated input cost: ${total_cost:.2f}")


Estimated total tokens used: 22438
Estimated cost: $0.67


### output tokens

In [14]:
PRICE_PER_1000_TOKENS = 0.00006  # Replace with the correct value for GPT-4

output_formatted_prompt = f"Reason: Grade:"
output_tokens = len(tokenizer.encode(output_formatted_prompt)) + 15
total_cost = output_tokens  * PRICE_PER_1000_TOKENS

# Print results
print(f"Estimated total output tokens used: {output_tokens}")
print(f"Estimated output cost: ${total_cost:.4f}")

Estimated total output tokens used: 19
Estimated output cost: $0.0011
